# ZTF Variable Marshal's API

In this notebook, we will programmatically interact with the ZTF Variable Marshal (ZVM) using its RESTful API. To simplify usage, the `zvm` `python` module

In [15]:
from IPython.core.display import display, HTML, JSON

from zvm import zvm
import json

For security, let us store the access credentials in a local `json` file `secrets_zvm.json`:

In [16]:
secrets = {
    "zvm": {
        "protocol": "http",
        "host": "localhost",
        "port": 8000,
        "username": "admin",
        "password": "admin"
    }
}

with open('secrets_zvm.json', 'w') as f:
    json.dump(secrets, f)

Load the credentials and initialize a `zvm` object:

In [17]:
with open('secrets_zvm.json', 'r') as f:
    secrets = json.load(f)

z = zvm(**secrets['zvm'], verbose=False)

## Interact with ZVM

### Programs

Get programs:

In [23]:
r = z.api(endpoint='programs', method='get', data={'format': 'json'})
display(JSON(r, expanded=True))

<IPython.core.display.JSON object>

Add new program:

In [21]:
r = z.api(endpoint='programs', method='put', data={'program_name': 'variables', 
                                                   'program_description': 'dummy program'})
display(JSON(r, expanded=True))

{'message': 'success',
 'result': {'_id': 3,
  'name': 'variables',
  'description': 'dummy program',
  'last_modified': datetime.datetime(2019, 10, 16, 0, 56, 6, 383000, tzinfo=<bson.tz_util.FixedOffset object at 0x106642b38>)}}

**todo:** `post` and `delete`

### Sources

Get `source_id` source data in `JSON` format:

In [25]:
source_id = 'ZTFS1922e'
r = z.api(endpoint=f'sources/{source_id}', method='get', data={'format': 'json'})
display(JSON(r, expanded=False))

<IPython.core.display.JSON object>

### Query sources saved on the ZVM

Note: The interface is *almost* exactly the same as in the `penquins` library. The source data are stored in the `sources` collection in ZVM's `mongodb`.

In [5]:
q = {"query_type": "find",
     "query": {
         "catalog": "sources",
         "filter": {},
         "projection": {'_id': 1}
     },
     "kwargs": {
         "sort": [('_id', -1)],
         "limit": 5 
     }
    }
r = z.query(query=q)
data = r['result']['result_data']['query_result']
# display(JSON(r, expanded=True))
display(JSON(data, expanded=True))

<IPython.core.display.JSON object>

In [6]:
q = {"query_type": "cone_search",
     "object_coordinates": {
         "radec": '[(332.202248, -2.0567986)]', 
         "cone_search_radius": "2",
         "cone_search_unit": "arcsec"
     },
     "catalogs": {
         "sources": {
             "filter": {
                 "zvm_program_id": {"$in": [1, 2]}
             },
             "projection": {
                 "_id": 1
             }
         }
     },
     "kwargs": {
         "limit": 3
     }
     }
r = z.query(query=q)
# r
data = r['result']['result_data']
display(JSON(data, expanded=True))

<IPython.core.display.JSON object>